In [1]:
from langchain_community.document_loaders import YoutubeLoader


In [2]:
loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=W1ZZ-Yni8Fg", add_video_info=True
)


In [3]:
data = loader.load()

In [4]:
data[0].metadata

{'source': 'W1ZZ-Yni8Fg',
 'title': "Why I'm embarrassed to be German",
 'description': 'Unknown',
 'view_count': 1721185,
 'thumbnail_url': 'https://i.ytimg.com/vi/W1ZZ-Yni8Fg/hq720.jpg',
 'publish_date': '2024-07-06 00:00:00',
 'length': 743,
 'author': 'Sabine Hossenfelder'}

In [5]:
data[0].page_content

"I never thought I'd say this but I'm embarrassed to be German things are going seriously wrong here in Germany and I want to talk about this because I think from the outside it's not all that obvious yet I've had many people from overseas asking me what happened to your country it didn't use to be that way and that's right Germany's not the well functioning environmentally superconscious Hightech Nation it used to be you go to a train station half the trains are cancelled the other half's delayed the escalators are broken the announcements are barely Audible and in the unlikely event that you can connect to the Wi-Fi it's so slow that by the time the news is streaming it's become a historical documentary or maybe it's Quantum Wi-Fi collapses the moment you touch it okay I'm exaggerating it's not always and everywhere that bad but it gives you an impression something's wrong and it's not just the railway the German government has made a long sequence of bad decisions about our infrastr

In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

def chunk_transcript(doc: Document) -> list[Document]:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=0,
        length_function=len,
        add_start_index=True,
    )

    chunks = text_splitter.split_documents([doc])

    # add end_index
    for chunk in chunks:
        chunk.metadata["end_index"] = chunk.metadata["start_index"] + len(
            chunk.page_content
        )

    return chunks

In [28]:
def generate_qa_from_transcript(transcript: Document) -> list[dict[str, str]]:

    chunks = chunk_transcript(transcript)

    return chunks

In [29]:
chunks = generate_qa_from_transcript(data[0])

In [33]:
chunks[0]

Document(metadata={'source': 'W1ZZ-Yni8Fg', 'title': "Why I'm embarrassed to be German", 'description': 'Unknown', 'view_count': 1721185, 'thumbnail_url': 'https://i.ytimg.com/vi/W1ZZ-Yni8Fg/hq720.jpg', 'publish_date': '2024-07-06 00:00:00', 'length': 743, 'author': 'Sabine Hossenfelder', 'start_index': 0, 'end_index': 990}, page_content="I never thought I'd say this but I'm embarrassed to be German things are going seriously wrong here in Germany and I want to talk about this because I think from the outside it's not all that obvious yet I've had many people from overseas asking me what happened to your country it didn't use to be that way and that's right Germany's not the well functioning environmentally superconscious Hightech Nation it used to be you go to a train station half the trains are cancelled the other half's delayed the escalators are broken the announcements are barely Audible and in the unlikely event that you can connect to the Wi-Fi it's so slow that by the time the 